In [1]:
"""
2^n
"""
def cut_rod(p, n):
    if n == 0:
        return 0
        
    q = float('-inf')
    
    for i in range(n):
        q = max(q, p[i] + cut_rod(p, n - 1))
        
    return q

In [4]:
def memoized_cut_rod(p, n):
    r = [float("-inf")] * (n + 1)
    r[0] = 0
    return memoized_cut_rod_aux(p, n, r)

def memoized_cut_rod_aux(p, n, r):
    if r[n] >= 0:
        return r[n]
        
    q = float("-inf")
    for i in range(n):
        q = max(q, p[i] + memoized_cut_rod_aux(p, n - i - 1, r))
        
    r[n] = q
    return q

def bottom_up_cut_rod(p, n):
    r = [0] * (n + 1)
    
    for j in range(1, n + 1):
        q = float("-inf")
        
        for i in range(j):
            q = max(q, p[i] + r[j - i - 1])
        r[j] = q
    
    return r[n]

In [8]:
def extended_bottom_up_cut_rod(p, n):
    r = [0] * (n + 1)
    s = [0] * (n + 1)
    
    for j in range(1, n + 1):
        q = float("-inf")
        for i in range(1, j + 1):
            value = p[i] + r[j - i]
            if q < value:
                q = value
                s[j] = i
        r[j] = q
    return r, s

def print_cut_rod_solution(p, n):
    (r, s) = extended_bottom_up_cut_rod(p, n)
    
    while n > 0:
        print(s[n])
        n = n - s[n]

In [9]:
"""
p = [0,1,5,8,9,10,17,17,20,24,30]
n = 7

optimal solution is:
Revenue = 18
Cuts: 1, 6

The `memoized_cut_rod_aux` starts at 0 to n
`memoized_cut_rod_aux_rev` Starts at n and goes backwards
"""
def memoized_cut_rod_rev(p, n):
    r = [-1] * (n + 1)
    s = [0] * (n + 1)

    revenue = memoized_cut_rod_aux(p, n, r, s)
    return revenue, s


def memoized_cut_rod_aux_rev(p, n, r, s):
    if r[n] >= 0:
        return r[n]

    if n == 0:
        q = 0
    else:
        q = float('-inf')

        for i in range(1, n + 1):
            value = p[i] + memoized_cut_rod_aux(p, n - i, r, s)
            if value > q:
                q = value
                s[n] = i

    r[n] = q
    return q

In [10]:
def fibonacci(n):
    if n <= 1:
        return n

    prev2 = 0
    prev1 = 1

    for _ in range(2, n + 1):
        current = prev1 + prev2
        prev2 = prev1
        prev1 = current

    return prev1

In [12]:
def rectangular_matrix_multiply(A, B, C, p, q, r):
    for i in range(p):
        for j in range(r):
            C[i][j] = 0
            for k in range(q):
                C[i][j] += A[i][k] * B[k][j]

def rectangular_matrix_multiply_cleaner(A, B):
    p = len(A)
    q = len(A[0])
    r = len(B[0])

    C = [[0 for _ in range(r)] for _ in range(p)]

    for i in range(p):
        for j in range(r):
            for k in range(q):
                C[i][j] += A[i][k] * B[k][j]

    return C

In [13]:
"""
A1 = 5×4
A2 = 4×6
A3 = 6×2

p = [5, 4, 6, 2]
m = [[0,0,0],
     [0,0,0],
     [0,0,0]]

i <= j (this forms a triangle)

m[0][0] → cost of A1 alone
m[0][1] → cost of A1A2
m[0][2] → cost of A1A2A3
     
"""
def matrix_chain_order(p):
    n = len(p) - 1
    
    m = [[0] * n for _ in range(n)]
    s = [[0] * n for _ in range(n)]
        
    for l in range(2, n + 1):
        for i in range(n - l + 1):
            j = i + l - 1
            m[i][j] = float("inf")
            
            for k in range(i, j):
                q = m[i][k] + m[k + 1][j] + p[i] * p[k + 1] * p[j + 1]
                
                if q < m[i][j]:
                    m[i][j] = q
                    s[i][j] = k
    return m, s